# DDoS dataset analysis

L'obiettivo di questo progetto è analizzare un _dataset_ contenente informazioni relative sia ad attacchi DDoS che a normale traffico.
Questo per poter realizzare un'applicazione capace di distinguere il traffico sospetto da quello legittimo e poter quindi tempestivamente bloccare i tentativi di attacco.

## Descrizione del dataset

Il _dataset_ è stato tratto da un _paper_ realizzato dalla "University of New South Wales", in Australia.
Il _paper_ è stato pubblicato su [Science Direct](https://www.sciencedirect.com/science/article/abs/pii/S0167739X18327687) ed è disponibile su [ResearchGate](https://www.researchgate.net/publication/328736466_Towards_the_Development_of_Realistic_Botnet_Dataset_in_the_Internet_of_Things_for_Network_Forensic_Analytics_Bot-IoT_Dataset) in _preprint_.
Come è scritto nello stesso, l'obiettivo del _paper_ era la realizzazione del _dataset_ stesso, rispettando le condizioni di massimo realismo possibile del traffico generato e della configurazione dell'ambiente in cui gli attacchi simulati sono stati svolti.

Il _paper_ indica anche il fatto che sono stati generate diverse tipologie di attacco, ma noi considereremo solamente quelle inerenti agli attacchi di tipo "Distributed Denial of Service", o "DDoS" in breve.
Il _software_ utilizzato per effettuare le catture dei pacchetti è stato [Argus](https://openargus.org/), la cui documentazione, nonché i suoi [esempi d'uso](https://openargus.org/using-argus), indicano come sono costruiti i _record_ che l'applicazione salva nel momento nel quale viene fatta una cattura di rete.

Ogni _record_ è il risultato di un raggruppamento di più pacchetti che svolgono la stessa funzione all'interno di una specifica connessione, o _flow_.
Ad esempio, un _record_ può contenere i pacchetti utilizzati dal protocollo "TCP" per effettuare l'_handshake_ con un'altro nodo di rete, il corpo della trasmissione, oppure la chiusura finale.
Per questo motivo ogni _record_, oltre a contenere informazioni capaci di identificare sorgente e destinazione della connessione, contengono anche dati derivanti dall'aggregazione sulle informazioni di più pacchetti.
Infine, dacché è possibile risalire dai _record_ alle singole connessioni, così come esplicitato nel _paper_ stesso, sono presenti anche già informazioni di aggregazione su alcuni parametri tra più record, informazioni che ci aspettiamo siano replicate uguali tra tutti i _record_ coinvolti.

Il sito in cui il _dataset_ è stato pubblicato è [questo](https://research.unsw.edu.au/projects/bot-iot-dataset), mentre il _download_ diretto dei file può essere fatto dalla [cartella](https://cloudstor.aarnet.edu.au/plus/s/umT99TnxvbpkkoE?path=%2FLabelling) di un servizio _cloud_ della UNSW.
Purtroppo, non è possibile effettuare il _download_ diretto dei file.
I _file_ che sono stati utilizzati in questo progetto sono quelli denominati "DDoS_HTTP.csv", "DDoS_TCP.csv" e "DDoS_UDP.csv".

### Descrizione dei file

I tre _file_ che sono stati utilizzati contengono tre diverse sotto-categorie di attacchi, ovvero attacchi che inviano messaggi "HTTP", segmenti "TCP" e datagrammi "UDP".
Non siamo interessati a tenere conto di questa distinzione, tanto più che tutti e tre i _file_ essendo stati generati dallo stesso _tool_, possiedono lo stesso formato, "CSV", e gli stessi campi.

I campi presenti in ciascun file sono i seguenti:

* "stime": la data e l'ora di ricezione del primo pacchetto del _record_
* "flgs": le _flag_ dello stato della connessione presenti nei pacchetti del _record_
* "proto": il protocollo di livello di rete utilizzato dai pacchetti del _record_
* "saddr": l'indirizzo IP dell'interfaccia sorgente dei pacchetti del _record_
* "sport": la porta dell'interfaccia sorgente dei pacchetti del _record_
* "dir": la direzione del flusso dati, da sorgente a destinazione, viceversa o bidirezionale
* "daddr": l'indirizzo IP dell'interfaccia destinazione dei pacchetti del _record_
* "dport": la porta dell'interfaccia destinazione dei pacchetti del _record_
* "pkts": il numero di pacchetti aggregati dal _record_
* "bytes": la somma dei _byte_ dei pacchetti aggregati
* "state": lo stato della connessione per i pacchetti aggregati dal _record_
* "srcid": l'identificatore usato dal _tool_ "Argus" per identificare la sorgente dati
* "ltime": la data e l'ora di ricezione dell'ultimo pacchetto del _record_
* "seq": il numero di sequenza che il _tool_ "Argus" ha assegnato al _record_
* "dur": la durata totale del _record_ 
* "mean": la durata media dei _record_ aggregati
* "stddev": la deviazione standard dei _record_ aggregati
* "smac": l'indirizzo MAC della sorgente dei pacchetti del _record_
* "dmac": l'indirizzo MAC della destinazione dei pacchetti del _record_
* "sum": la somma delle durate dei _record_ aggregati
* "min": il minimo delle durate dei _record_ aggregati_
* "max": il massimo delle durate dei _record_ aggregati
* "soui": lo "Organizationally Unique Identifier" dell'indirizzo MAC della sorgente dei pacchetti del _record_
* "doui": lo "Organizationally Unique Identifier" dell'indirizzo MAC della destinazione dei pacchetti del _record_
* "sco": il "Country Code" associato all'indirizzo IP della sorgente dei pacchetti nel _record_
* "dco": il "Country Code" associato all'indirizzo IP della destinazione dei pacchetti nel _record_
* "spkts": il numero di pacchetti inviati dalla sorgente alla destinazione in questo _record_
* "dpkts": il numero di pacchetti inviati dalla destinazione alla sorgente in questo _record_
* "sbytes": il numero di _byte_ inviati dalla sorgente alla destinazione in questo _record_
* "dbytes": il numero di _byte_ inviati dalla destinazione alla sorgente in questo _record_
* "rate": i pacchetti al secondo inviati in questo _record_
* "srate": i pacchetti al secondo inviati dalla sorgente alla destinazione in questo _record_
* "drate": i pacchetti al secondo inviati dalla destinazione alla sorgente in questo _record_
* "record": questa feature non è spiegata all'interno del _paper_ né tantomeno nella documentazione di "Argus"
* "attack: se il _record_ è parte di un attacco o meno
* "category": la categoria dell'attacco
* "subcategory": la specifica sotto-categoria dell'attacco

I campi che abbiamo utilizzato nell'analisi sono stati:

* stime
* proto
* saddr
* sport
* dir
* daddr
* dport
* pkts
* bytes
* ltime
* dur
* spkts
* dpkts
* sbytes
* dbytes
* rate
* srate
* drate
* attack

## Preparazione dei dati

Per effettuare la preparazione dei dati, innanzitutto configuriamo il _kernel_ Spark che utilizzeremo.

In [1]:
%%configure -f
{
    "executorMemory": "8G", 
    "numExecutors": 3, 
    "executorCores": 3, 
    "conf": {
        "spark.dynamicAllocation.enabled": "false"
    }, 
    "jars": [
        "s3a://unibo-bd2122-mcastellucci/project/evilplot-repl_2.12-0.8.1.jar",
        "s3a://unibo-bd2122-mcastellucci/project/evilplot_2.12-0.8.1.jar",
        "s3a://unibo-bd2122-mcastellucci/project/circe-core_2.12-0.13.0.jar",
        "s3a://unibo-bd2122-mcastellucci/project/circe-parser_2.12-0.13.0.jar",
        "s3a://unibo-bd2122-mcastellucci/project/circe-generic_2.12-0.13.0.jar",
        "s3a://unibo-bd2122-mcastellucci/project/circe-generic-extras_2.12-0.13.0.jar",
        "s3a://unibo-bd2122-mcastellucci/project/scalactic_2.12-3.0.8.jar"
    ]
}

Dopodiché, definiamo i percorsi dei file che utilizzeremo così come sono stati salvati sul servizio "Amazon S3" ed avviamo una nuova applicazione Spark.

In [2]:
val pathTCPDataset = s"s3a://$bucketName/DDoS_TCP.csv"
val pathUDPDataset = s"s3a://$bucketName/DDoS_UDP_original.csv"
val pathHTTPDataset = s"s3a://$bucketName/DDoS_HTTP.csv"

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
10,application_1655730958384_0011,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketName: String = unibo-bd2122-mcastellucci/project
pathTCPDataset: String = s3a://unibo-bd2122-mcastellucci/project/DDoS_TCP.csv
pathUDPDataset: String = s3a://unibo-bd2122-mcastellucci/project/DDoS_UDP_original.csv
pathHTTPDataset: String = s3a://unibo-bd2122-mcastellucci/project/DDoS_HTTP.csv
res3: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1655730958384_0011/


A questo punto è possibile costruire l'RDD per intero, in modo tale che contenga i dati di tutti e tre i file che ci interessano.

In [18]:
val dataset = sc.textFile(s"$pathTCPDataset,$pathUDPDataset,$pathHTTPDataset")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dataset: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2122-mcastellucci/project/DDoS_TCP.csv,s3a://unibo-bd2122-mcastellucci/project/DDoS_UDP_original.csv,s3a://unibo-bd2122-mcastellucci/project/DDoS_HTTP.csv MapPartitionsRDD[1] at textFile at <console>:31


A questo punto si tratta di fare _parsing_ dei _record_ del _dataset_.
Per effettuarlo correttamente, teniamo conto delle seguenti informazioni sui formati dei valori nelle singole colonne:

* stime, ltime: il valore è un _timestamp_ in secondi dall'epoca UNIX, anche se è espresso in formato decimale per poter avere la precisione dei millisecondi
* proto: il valore può essere uno tra "tcp", "udp", "arp" e "icmp-v6", ognuno dei quali è associato al corrispondente protocollo, sono però di interesse solamente i _record_ associati ai protocolli TCP e UDP
* saddr, dadd: il valore è un indirizzo IP in formato "dotted decimal notation", può perciò essere salvato come String
* sport, dport: il valore è un intero positivo che può arrivare ad un massimo di 65.536, perciò per poter essere rappresentato in linguaggio scala necessita di essere salvato in un Long
* dir:
* pkts, bytes, spkts, dpkts, sbytes, dbytes: il valore è un intero positivo di cui non è noto il massimo, per cui è logico pensare di salvare il valore in un Long
* dur, rate, srate, drate: il valore è un numero decimale, perciò per mantenere la precisione massima utilizziamo un Double
* attack: il valore può essere "1" nel caso il _record_ appartenga ad un attacco DDoS, "0" in caso contrario

Detto questo, sono state implementati i seguenti Astract Data Types:

In [19]:
import java.net.InetAddress
import java.time.format.{ DateTimeFormatter, DateTimeParseException }
import java.time.{ Instant, LocalDateTime, ZoneId }
import scala.util.{ Try, Success, Failure }
import org.apache.spark.HashPartitioner

object NetworkProtocol extends Enumeration {
  type NetworkProtocol = Value

  val TCP: NetworkProtocol = Value
  val UDP: NetworkProtocol = Value
}

import NetworkProtocol.NetworkProtocol

case class Record(
    startTime: LocalDateTime,
    protocol: NetworkProtocol,
    sourceAddress: String,
    sourcePort: Long,
    direction: String,
    destinationAddress: String,
    destinationPort: Long,
    packets: Long,
    bytes: Long,
    endTime: LocalDateTime,
    duration: Double,
    sourceBytes: Long,
    destinationBytes: Long,
    rate: Double,
    sourceRate: Double,
    destinationRate: Double,
    isDDoS: Boolean
)

object Record {

  def apply(r: Seq[String]): Option[Record] = 
    (for {
      startTime <- Try(Instant.ofEpochMilli((r.head.toDouble * 1000).toLong).atZone(ZoneId.systemDefault()).toLocalDateTime)
      protocol <- if (r(2) == "tcp") Success(NetworkProtocol.TCP) else if (r(2) == "udp") Success(NetworkProtocol.UDP) else Failure(new IllegalStateException())
      sourceAddress = r(3)
      sourcePort <- Try(r(4).toLong)
      direction = r(5)
      destinationAddress = r(6)
      destinationPort <- Try(r(7).toLong)
      packets <- Try(r(8).toLong)
      bytes <- Try(r(9).toLong)
      endTime <- Try(Instant.ofEpochMilli((r(12).toDouble * 1000).toLong).atZone(ZoneId.systemDefault()).toLocalDateTime)
      duration <- Try(r(14).toDouble)
      sourceBytes <- Try(r(28).toLong)
      destinationBytes <- Try(r(29).toLong)
      rate <- Try(r(30).toDouble)
      sourceRate <- Try(r(31).toDouble)
      destinationRate <- Try(r(32).toDouble)
      isDDoS = r(34) == "1"
     } yield new Record(
         startTime,
         protocol,
         sourceAddress,
         sourcePort,
         direction,
         destinationAddress,
         destinationPort,
         packets,
         bytes,
         endTime,
         duration,
         sourceBytes,
         destinationBytes,
         rate,
         sourceRate,
         destinationRate,
         isDDoS
       )
    ).toOption
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.net.InetAddress
import java.time.format.{DateTimeFormatter, DateTimeParseException}
import java.time.{Instant, LocalDateTime, ZoneId}
import scala.util.{Try, Success, Failure}
import org.apache.spark.HashPartitioner
defined object NetworkProtocol
import NetworkProtocol.NetworkProtocol
defined class Record
defined object Record
Companions must be defined together; you may wish to use :paste mode for this.


Alla definizione è seguito il parsing vero e proprio, che ha tenuto conto del fatto che i tre _file_, essendo in formato CSV, hanno le virgolette che circondano ogni valore di ogni colonna e sono separati dai punti e virgola.
Assieme ai _record_ "legittimi", per così dire, saranno presenti anche le intestazioni dei tre _file_.
Questo però non ci preoccupa perché sappiamo che il _parser_ eliminerà correttamente quelle righe dall'RDD che caricheremo, non avendo lo stesso formato delle altre.
Effettuiamo il _caching_ del _dataset_ perché lo riutilizzeremo più volte e così riusciamo a vedere la sua occupazione in formato non serializzato.
La quantità di memoria occupata è di 8.1GB.

In [20]:
val recordDataset = 
    dataset.
        map(_.replace("\"", "")).
        map(_.split(";")).
        map(Record(_)).
        filter(_.isDefined).
        map(_.get).
        cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

recordDataset: org.apache.spark.rdd.RDD[Record] = MapPartitionsRDD[6] at map at <console>:41


Il dataset così caricato e ripulito è formato da 38.532.503 _record_.

In [21]:
val recordDatasetSize = recordDataset.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

recordDatasetSize: Long = 38532503


## Query

### Percentuale di _record_ associati ad attacchi DDoS nell'intero _dataset_

Con questa query si vuole calcolare la percentuale di _record_ che appartengono ad attacchi DDoS nel _dataset_ e così derivare anche il numero di _record_ che __non__ appartengono ad attacchi DDoS, ma a traffico legittimo.

In [22]:
val ddosCount = 
    recordDataset.
        map(r => if (r.isDDoS) (1, 0) else (0, 1)).
        reduce{ case((ddosCount1, legitCount1), (ddosCount2, legitCount2)) => (ddosCount1 + ddosCount2, legitCount1 + legitCount2) }

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ddosCount: (Int, Int) = (38531238,1265)


La query prende in input l'intero _dataset_ e produce due Long: il primo è il numero di _record_ appartenenti ad attacchi DDoS, il secondo è il numero di _record_ appartenenti a traffico legittimo.

La query ha completato in media con un tempo di 52 secondi prendendo in input 12.7 GB di dati.

L'implementazione scelta non fa altro che trasformare ogni _record_ in una coppia di valori, dove il primo rappresenta il conteggio dei _record_ associati ad attacchi, il secondo a traffico legittimo, che avrà il valore 1 nella posizione corrispondente a quale tipo di traffico il _record_ stesso appartiene, il valore 0 nell'altra.
Dopodiché, viene compiuta un'operazione di _reduce_ che somma i valori nelle corrispondenti posizioni.

È stata tentata una variante ottimizzata dove si raggruppavano i _record_ per tipologia di traffico di appartenenza, ma aveva un tempo di esecuzione comparabile a questa implementazione, pur raddoppiando il numero di _task_, dopodiché sulla nuova implementazione si è tentato di forzare il partizionamento a 2 partizioni tramite `HashPartioner`, ma ha peggiorato il tempo di esecuzione.
Evidentemente lo _shuffling_ derivato dal ripartizionamento non è giustificato da un eventuale riduzione del tempo di esecuzione della _reduce_.

In [23]:
val ddosPercentage = ddosCount._1 / recordDatasetSize.toDouble * 100
val legitPercentage = ddosCount._2 / recordDatasetSize.toDouble * 100

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ddosPercentage: Double = 99.99671705728538
legitPercentage: Double = 0.0032829427146219906


### Conteggio dei flussi catturati nel dataset
Un flusso lo si identifica da (sourceIp, sourcePort, destinationIp, destinationPort)

In [ ]:
val flowsDataset = recordDataset.
    groupBy(r => (r.saddr, r.sport, r.daddr, r.dport)).cache()

In [ ]:
val flowsCount = flowsDataset.count()

In [ ]:
val ddosFlowsCount = flowsDataset.filter { case (_, packets) => packets.forall(_.isDDoS) }.count()
val legitFlowsCount = flowsDataset.filter { case (_, packets) => packets.forall(!_.isDDoS) }.count()

Determinati i flussi, possiamo calcolare mediamente quanti pacchetti sono scambiati per ogni flusso:

In [ ]:
val meanPacketPerFlow = datasetSize / flowsCount.toDouble
val meanPacketDDoSFlow = ddosVolume / ddosFlowsCount.toDouble
val meanPacketLegitFlow = legitVolume / legitFlowsCount.toDouble